In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import seaborn as sns
from scipy.stats import ttest_ind

df = pd.read_csv("../Speed_dating.csv", encoding='ISO-8859-1')

In [2]:
# Liste des attributs analysés
attributes = ['attr', 'sinc', 'intel', 'fun', 'amb']

# Initialiser les résultats
detailed_diffs = []

# Calculer les écarts pour chaque attribut
for attr in attributes:
    col_self = f'{attr}3_1'         # Auto-évaluation
    col_think_others = f'{attr}5_1' # Perception supposée
    col_received = attr             # Note réelle pendant les dates

    # Extraire les colonnes utiles et supprimer les NaN
    valid_df = df[['iid', col_self, col_think_others, col_received]].dropna()

    # Moyenne des notes reçues pendant les dates (réel)
    received_avg = df.groupby('iid')[col_received].mean().rename("received")
    valid_df = valid_df.set_index('iid').join(received_avg, rsuffix='_avg', how='inner')

    # Calcul des écarts absolus
    valid_df['self_vs_other'] = (valid_df[col_self] - valid_df['received']).abs()
    valid_df['self_vs_perceived'] = (valid_df[col_self] - valid_df[col_think_others]).abs()
    valid_df['perceived_vs_other'] = (valid_df[col_think_others] - valid_df['received']).abs()

    # Stocker les moyennes des écarts
    detailed_diffs.append({
        'Attribut': attr,
        'Écart moyen – Moi vs Note réelle': round(valid_df['self_vs_other'].mean(), 2),
        'Écart moyen – Moi vs Ce que je pense que les autres pensent': round(valid_df['self_vs_perceived'].mean(), 2),
        'Écart moyen – Ce que je pense que les autres pensent vs Note réelle': round(valid_df['perceived_vs_other'].mean(), 2)
    })

# Création du DataFrame récapitulatif
diffs_df = pd.DataFrame(detailed_diffs)
print(diffs_df)


  Attribut  Écart moyen – Moi vs Note réelle  \
0     attr                              1.56   
1     sinc                              1.69   
2    intel                              1.40   
3      fun                              1.74   
4      amb                              1.75   

   Écart moyen – Moi vs Ce que je pense que les autres pensent  \
0                                               0.44             
1                                               0.88             
2                                               0.60             
3                                               0.75             
4                                               1.08             

   Écart moyen – Ce que je pense que les autres pensent vs Note réelle  
0                                               1.63                    
1                                               1.55                    
2                                               1.43                    
3                     

In [3]:
import plotly.express as px

fig = px.bar(
    diffs_df.melt(id_vars='Attribut', var_name='Type d\'écart', value_name='Écart moyen'),
    x='Attribut',
    y='Écart moyen',
    color='Type d\'écart',
    barmode='group',
    title="Écarts entre perception de soi, perception supposée et réalité (Time1)",
    template='plotly_white'
)

fig.show()


In [7]:
import pandas as pd

# Liste des attributs à évaluer
attributes = ['attr', 'sinc', 'intel', 'fun', 'amb']

# Résultats stockés ici
time1_results = []

# Boucle sur chaque attribut
for attr in attributes:
    col_self = f"{attr}3_1"       # Auto-évaluation
    col_perceived = f"{attr}5_1"  # Ce que je pense que les autres pensent
    col_real = attr               # Note réelle reçue pendant les speed-dates

    # Vérifier que les colonnes existent
    if all(col in df.columns for col in [col_self, col_perceived, col_real]):
        # Extraire les données propres
        sub_df = df[['iid', col_self, col_perceived]].dropna().drop_duplicates(subset='iid').set_index('iid')

        # Ajouter la note réelle moyenne reçue pendant les speed-dates
        received = df.groupby('iid')[col_real].mean().rename("real_received")
        sub_df = sub_df.join(received, how='inner')

        # Calcul des écarts absolus
        sub_df['self_vs_perceived'] = (sub_df[col_self] - sub_df[col_perceived]).abs()
        sub_df['self_vs_real'] = (sub_df[col_self] - sub_df['real_received']).abs()
        sub_df['perceived_vs_real'] = (sub_df[col_perceived] - sub_df['real_received']).abs()

        # Stocker les moyennes
        time1_results.append({
            'Attribut': attr,
            'Type d\'écart': 'Moi vs Ce que je pense que les autres pensent',
            'Écart moyen': round(sub_df['self_vs_perceived'].mean(), 2)
        })
        time1_results.append({
            'Attribut': attr,
            'Type d\'écart': 'Moi vs Note réelle',
            'Écart moyen': round(sub_df['self_vs_real'].mean(), 2)
        })
        time1_results.append({
            'Attribut': attr,
            'Type d\'écart': 'Ce que je pense que les autres pensent vs Note réelle',
            'Écart moyen': round(sub_df['perceived_vs_real'].mean(), 2)
        })

# Convertir en DataFrame final
time1_df = pd.DataFrame(time1_results)
print(time1_df)


   Attribut                                       Type d'écart  Écart moyen
0      attr      Moi vs Ce que je pense que les autres pensent         0.45
1      attr                                 Moi vs Note réelle         1.55
2      attr  Ce que je pense que les autres pensent vs Note...         1.63
3      sinc      Moi vs Ce que je pense que les autres pensent         0.89
4      sinc                                 Moi vs Note réelle         1.67
5      sinc  Ce que je pense que les autres pensent vs Note...         1.53
6     intel      Moi vs Ce que je pense que les autres pensent         0.60
7     intel                                 Moi vs Note réelle         1.39
8     intel  Ce que je pense que les autres pensent vs Note...         1.40
9       fun      Moi vs Ce que je pense que les autres pensent         0.75
10      fun                                 Moi vs Note réelle         1.69
11      fun  Ce que je pense que les autres pensent vs Note...         1.69
12      amb 

In [6]:
import plotly.express as px

# Filtrer uniquement Time1
time1_df = extended_df[extended_df["Moment"] == "Time1"]

# Créer le graphique
fig = px.bar(
    time1_df,
    x="Attribut",
    y="Écart moyen",
    color="Type d'écart",
    barmode="group",
    title="Écarts de perception – Time1 uniquement (auto-évaluation, supposée, réelle)",
    labels={"Écart moyen": "Écart absolu moyen"},
    height=500,
    template="plotly_white"
)
fig.update_layout(xaxis_tickangle=45)
fig.show()
